In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
import joblib
import tabulate as tb
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import Huber
from tensorflow.keras import Sequential, layers, optimizers, losses
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import os, random, numpy as np, tensorflow as tf
from model import FinancialLSTMModel
from tensorflow.keras.layers import GlobalAveragePooling1D
from custom_attention import CustomAttention

SEED = 42
os.environ["PYTHONHASHSEED"]=str(SEED)
os.environ["TF_DETERMINISTIC_OPS"]="1"
os.environ["TF_CUDNN_DETERMINISTIC"]="1"
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
tf.config.threading.set_inter_op_parallelism_threads(1)
tf.config.threading.set_intra_op_parallelism_threads(1)

2025-11-30 21:22:27.999991: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-30 21:22:28.085169: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-30 21:22:30.164642: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


In [2]:
CSV_PATH = './../data/AAPL_1h.csv'
DATE_COL = 'Datetime'

SEQ_LENGTH = 90
BATCH_SIZE = 32
LEARNING_RATE = 0.00005
EPOCHS = 100
TEST_RATIO = 0.2
VAL_SPLIT = 0.1

REPS = 3

EXCLUDE_COLUMNS = ['Datetime', 'returns', 'direction'] 
FEATURES = []

TARGET = 'direction'

def build_hidden_layers1():
    return [
        tf.keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        
        tf.keras.layers.MaxPooling1D(pool_size=2),
        
        tf.keras.layers.LSTM(128, return_sequences=True, activation='tanh'),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.BatchNormalization(),
        
        tf.keras.layers.LSTM(64, return_sequences=True, activation='tanh'),
        tf.keras.layers.Dropout(0.4),
        
        CustomAttention(name='attention_layer'),
        
        GlobalAveragePooling1D(name='context_vector_aggregation'),
    ]


In [3]:
if len(FEATURES) == 0:
    data = pd.read_csv(CSV_PATH, parse_dates=[DATE_COL])
    all_cols = data.columns.tolist()
    FEATURES = [(feat, "minmax") for feat in all_cols if feat not in EXCLUDE_COLUMNS]
    print("Using features:", FEATURES)

Using features: [('Close', 'minmax'), ('High', 'minmax'), ('Low', 'minmax'), ('Open', 'minmax'), ('Volume', 'minmax'), ('rsi_14', 'minmax'), ('rsi_28', 'minmax'), ('rsi_50', 'minmax'), ('rsi_7', 'minmax'), ('macd', 'minmax'), ('ema_10', 'minmax'), ('ema_20', 'minmax'), ('ema_50', 'minmax'), ('ema_100', 'minmax'), ('ema_200', 'minmax'), ('stoch_k', 'minmax'), ('stoch_d', 'minmax'), ('roc', 'minmax'), ('adx', 'minmax'), ('di_plus', 'minmax'), ('di_minus', 'minmax'), ('atr_14', 'minmax'), ('atr_20', 'minmax'), ('close_pos', 'minmax'), ('body_range_ratio', 'minmax'), ('volume_zscore_50', 'minmax'), ('bb_lower_20', 'minmax'), ('bb_middle_20', 'minmax'), ('bb_upper_20', 'minmax'), ('bb_width_20', 'minmax'), ('obv', 'minmax'), ('atr_5', 'minmax'), ('log_returns', 'minmax'), ('rolling_max_20', 'minmax'), ('rolling_min_20', 'minmax'), ('price_from_20d_high', 'minmax')]


In [4]:
res = []

for r in range(REPS):
    print(f"--- REPETITION {r+1}/{REPS} ---")
    model = FinancialLSTMModel(
        csv_path=CSV_PATH,
        features_scales=FEATURES,
        target_col="direction",
        datetime_col="Datetime",

        seq_length=SEQ_LENGTH,
        batch_size=BATCH_SIZE,
        learning_rate=LEARNING_RATE,
        epochs=EPOCHS,
        test_ratio=TEST_RATIO,
        val_split=VAL_SPLIT,
        weight_adj_factors=[1,1,1],
        under_sample_imbalanced=True
    )

    model.prepare_data()
    model.build_model(build_hidden_layers1())
    model.train()
    ev = model.evaluate()
    res.append(ev)
    
    print(f">> results: {ev}")

--- REPETITION 1/3 ---


2025-11-30 21:22:31.964405: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Epoch 1/100


2025-11-30 21:22:32.409731: E tensorflow/core/framework/node_def_util.cc:680] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_16}}


33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - auc_roc: 0.5203 - balanced_accuracy: 0.3291 - loss: 1.0982

...

>> results: {'accuracy': 0.11285266457680251, 'f1_score': 0.022888427745154313, 'balanced_accuracy': 0.3333333333333333, 'precision': np.float32(0.0), 'recall': np.float32(0.0), 'confusion_matrix': array([[ 72,   0,   0],
       [480,   0,   0],
       [ 86,   0,   0]])}
--- REPETITION 3/3 ---
Epoch 1/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - auc_roc: 0.4818 - balanced_accuracy: 0.3325 - loss: 1.0989

2025-11-30 21:26:13.746691: E tensorflow/core/framework/node_def_util.cc:680] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_15}}


33/33 ━━━━━━━━━━━━━━━━━━━━ 10s 114ms/step - auc_roc: 0.4858 - balanced_accuracy: 0.3254 - loss: 1.0989 - val_auc_roc: 0.5000 - val_balanced_accuracy: 0.3320 - val_loss: 1.0986 - learning_rate: 5.0000e-05
Epoch 2/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 88ms/step - auc_roc: 0.5205 - balanced_accuracy: 0.3564 - loss: 1.0982 - val_auc_roc: 0.5000 - val_balanced_accuracy: 0.3333 - val_loss: 1.0991 - learning_rate: 5.0000e-05
Epoch 3/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 82ms/step - auc_roc: 0.5373 - balanced_accuracy: 0.4005 - loss: 1.0977 - val_auc_roc: 0.5000 - val_balanced_accuracy: 0.3333 - val_loss: 1.0996 - learning_rate: 5.0000e-05
Epoch 4/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 82ms/step - auc_roc: 0.5619 - balanced_accuracy: 0.4025 - loss: 1.0971 - val_auc_roc: 0.5000 - val_balanced_accuracy: 0.3298 - val_loss: 1.1002 - learning_rate: 5.0000e-05
Epoch 5/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 82ms/step - auc_roc: 0.5663 - balanced_accuracy: 0.4255 - loss: 1.0967 - val_auc_roc: 0.4294 - val_balanced_accu

2025-11-30 21:27:48.323847: E tensorflow/core/framework/node_def_util.cc:680] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


>> results: {'accuracy': 0.219435736677116, 'f1_score': 0.22438379740610978, 'balanced_accuracy': 0.3451388888888889, 'precision': np.float32(0.90816325), 'recall': np.float32(0.15724382), 'confusion_matrix': array([[ 63,   9,   0],
       [401,  77,   2],
       [ 76,  10,   0]])}


In [5]:
df = pd.DataFrame(res)

print("\n=== SUMMARY ===")
print(tb.tabulate(df, headers='keys', tablefmt='pretty', showindex="always"))


=== SUMMARY ===
+---+---------------------+----------------------+--------------------+--------------------+---------------------+------------------+
|   |      accuracy       |       f1_score       | balanced_accuracy  |     precision      |       recall        | confusion_matrix |
+---+---------------------+----------------------+--------------------+--------------------+---------------------+------------------+
| 0 | 0.11285266457680251 | 0.022888427745154313 | 0.3333333333333333 |        0.0         |         0.0         |  [[ 72   0   0]  |
|   |                     |                      |                    |                    |                     |   [480   0   0]  |
|   |                     |                      |                    |                    |                     |  [ 86   0   0]]  |
| 1 | 0.11285266457680251 | 0.022888427745154313 | 0.3333333333333333 |        0.0         |         0.0         |  [[ 72   0   0]  |
|   |                     |                  